In [1]:
# 재귀에서 반복문으로
import numpy as np
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()  # 1. Get a function 리스트의 마지막 원소를 꺼낸다.
            x, y = f.input, f.output  # 2. Get the function's input/output f의 input 과 f의 output 을 꺼낸다. 둘다 variable 객체임 재귀호출이 아니기 때문에 f.output필요
            x.grad = f.backward(y.grad)  # 3. Call the function's backward output 객체 안에 저장된 grad 값을 꺼낸다.

            if x.creator is not None: # 추가한다. x.creator 꺼내기 -> 추가하기 -> 꺼내기 ... 계속 반복된다.
                funcs.append(x.creator)

재귀는 함수를 재귀적으로 호출할 때마다 중간 결과를 메모리에 쌓으면서 처리를 이어간다. 그래서 반복문이 더 효율 좋음(물론 요즘 컴퓨터는 메모리가 넉넉하여 문제가 되지 않음?)

In [5]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        print(self)
        self.input = input
        self.output = output

        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [6]:
A = Square()
B = Exp()
C = Square()


In [7]:
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [8]:
y.grad = np.array(1.0) # 처음 y grad 값은 없기 때문에 넣어준다.
y.backward()
print(x.grad)

3.297442541400256
